In [13]:
import os
str1 = ''
file_list = []

os.chdir(r"G:\NT数据\1\银行2")
for path_name, dir, files_name in os.walk(os.getcwd()):
    for file in files_name:
        if str1 in file:
            file_list.append(os.path.join(path_name, file))
            
print(len(file_list))
print('\n'.join(file_list))



import pandas as pd
df_group = pd.DataFrame(columns = ['Path','Columns','False','True'])
type1 = '.xls'
type2 = '.csv'
type3 = '.et'
output_path = 'G:\\'
if not os.path.exists(output_path + './资金流水'):
    os.mkdir(output_path + './资金流水')
for i in range(len(file_list)):
    if type1 in file_list[i]:
        df = pd.read_excel(file_list[i], encoding = "gbk")
        df = df.loc[:,~df.columns.str.contains('^Unnamed')]
    elif type2 in file_list[i]:
        df = pd.read_csv(file_list[i], encoding = 'GB18030')
        df = df.loc[:,~df.columns.str.contains('^Unnamed')]
    else:
        continue
    print(i ,end = ' ')
    df.columns = df.columns.str.strip()
    col_name = df.columns
    bool_S = df.duplicated().value_counts()
    if len(bool_S)==1:
        df_group.loc[i] = {'Path':file_list[i],'Columns':tuple(col_name),'False':bool_S[0],'True':0}
    elif len(bool_S)==2:
        df_group.loc[i] = {'Path':file_list[i],'Columns':tuple(col_name),'False':bool_S[0],'True':bool_S[1]}
grouped = df_group.groupby('Columns')
groupname_list = list(grouped.size().index)
print('以下'+str(len(groupname_list))+'种格式数据分类，保留流水、账户类别')
print(grouped.size())

306
G:\NT数据\1\银行2\.DS_Store
G:\NT数据\1\银行2\2.zip
G:\NT数据\1\银行2\1\3206270120200624112106.zip
G:\NT数据\1\银行2\1\3206270120200624112716.zip
G:\NT数据\1\银行2\1\3206270120200624141028.zip
G:\NT数据\1\银行2\1\3206270120200624142131.zip
G:\NT数据\1\银行2\1\3206270120200624142347.zip
G:\NT数据\1\银行2\1\3206270120200624142604.zip
G:\NT数据\1\银行2\1\3206270120200624143156.zip
G:\NT数据\1\银行2\1\3206270120200624143321.zip
G:\NT数据\1\银行2\1\3206270120200624143554.zip
G:\NT数据\1\银行2\1\3206270120200624143755.zip
G:\NT数据\1\银行2\1\3206270120200624144142.zip
G:\NT数据\1\银行2\1\3206270120200624144654.zip
G:\NT数据\1\银行2\1\3206270120200624145849.zip
G:\NT数据\1\银行2\1\3206270120200624112106\中信银行交易明细信息.csv
G:\NT数据\1\银行2\1\3206270120200624112106\中信银行人员信息.csv
G:\NT数据\1\银行2\1\3206270120200624112106\中信银行账户信息.csv
G:\NT数据\1\银行2\1\3206270120200624112106\人员住址信息.csv
G:\NT数据\1\银行2\1\3206270120200624112106\人员联系方式信息.csv
G:\NT数据\1\银行2\1\3206270120200624112106\任务信息(失败).csv
G:\NT数据\1\银行2\1\3206270120200624112106\任务信息(成功).csv
G:\NT数据\1\银行2\1\3206270120200

15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 

KeyboardInterrupt: 

In [ ]:
group_流水 = grouped.get_group(groupname_list[0])
group_流水['文件序号I'] = group_流水.index

group_证件= grouped.get_group(groupname_list[2])
group_证件2= grouped.get_group(groupname_list[3])
group_证件3= grouped.get_group(groupname_list[4])

group_账户 = grouped.get_group(groupname_list[8])
group_账户['文件序号I'] = group_账户.index

# group_流水2 = grouped.get_group(groupname_list[7])
# group_流水2['文件序号I'] = group_流水2.index

group_任务 = grouped.get_group(groupname_list[1])


with pd.ExcelWriter(output_path + "文件目录.xlsx") as writer:
    group_流水.to_excel(writer, sheet_name = '流水文件目录',index = None)
#     group_流水2.to_excel(writer, sheet_name ='流水文件目录-标准',index = None)
    group_账户.to_excel(writer, sheet_name ='账户文件目录',index = None)
print('文件目录Done!')


import numpy as np
import re
import datetime
df_忽略行 = pd.DataFrame()
df_冲正 = pd.DataFrame()
all_流水 = pd.DataFrame()
all_账户 = pd.DataFrame()
all_证件 = pd.DataFrame()
all_任务 = pd.DataFrame()
for i in range(len(file_list)):
    if file_list[i] in list(group_流水['Path']):
        if file_list[i] in list(group_流水['Path']):  #查控平台流水
            if type1 in file_list[i] or type3 in file_list[i]:
                df = pd.read_excel(file_list[i],encoding = "gbk",dtype = "object")
            elif type2 in file_list[i]:
                df = pd.read_csv(file_list[i], encoding = 'GB18030')
            df = df.loc[:,~df.columns.str.contains('^Unnamed')]
            df.columns = df.columns.str.strip()      # 去掉标题行的\t
            col_name = df.columns.tolist()
        print(i,end=' ')
        df['path'] = file_list[i]
        df['序号'] = df.index
        df = df.applymap(lambda x: re.sub('\t','',str(x)).strip())  # 去掉每个元素的\t
        df['交易卡号'] = df['交易卡号'].map(lambda x: np.NaN if str(x)=='' or str(x)==' ' else x)
        df['交易卡号'][df['交易卡号'].isnull()]=df['交易账号'][df['交易卡号'].isnull()] # 用账号填充卡号空白
        df_忽略行 = df_忽略行.append(df[(df['交易金额']=='') | (df['交易金额']=='交易金额') | (df['交易卡号']=='')| (df['交易时间']=='')])
        df = df.drop(df[(df['交易金额']=='') | (df['交易金额']=='交易金额') | (df['交易卡号']=='') | (df['交易时间']=='')].index)  # 记录后删除交易金额为空或者账号卡号都为空的无效流水行
        df['交易金额'] = df['交易金额'].map(lambda x: float(x))  #交易金额指定浮点型
        df['交易余额'] = df['交易余额'].map(lambda x: np.NaN if str(x)=='' else x)
        df['交易余额'] = df['交易余额'].map(lambda x: float(x))  #交易余额指定浮点型
        df['收付标志'] = df['收付标志'].map(lambda x:'出' if x=='借' else ('进' if x=='贷' else x))
        df = df.drop_duplicates(subset = ['交易卡号','交易时间','交易金额','交易余额','收付标志','交易对手账卡号'],keep = 'last')  #去除重复
        df = df.sort_values(by = ['交易卡号','交易时间','交易金额'],ascending=[True,True,False]).reset_index().iloc[:,1:] # 时间升序排序更新index
        print(str(i)+'. 单表去重后记录数:'+str(df.shape[0]),end = ' ')
        df_负 = df[df['交易金额']<0]
        for j in range(0,len(df_负)):
            df_正 = df[(df['交易金额']== abs(df_负['交易金额'].iloc[j])) & \
                (df['收付标志']== df_负['收付标志'].iloc[j]) & \
                (df['交易时间']<= df_负['交易时间'].iloc[j]) & \
                (df['交易卡号']== df_负['交易卡号'].iloc[j]) & \
                (
                 (df['交易对手账卡号']== df_负['交易对手账卡号'].iloc[j]) | \
                 ((df['交易对手账卡号'] == '') & (df_负['交易对手账卡号'].iloc[j] == '')) | \
                 ((df['交易对手账卡号'] != '') & (df_负['交易对手账卡号'].iloc[j] == '')) | \
                 ((df['交易对手账卡号'] == '') & (df_负['交易对手账卡号'].iloc[j] != '')) | \
                 ((df['交易对手账卡号'] != '') & (df['交易卡号']==df_负['交易对手账卡号'].iloc[j]))
                )
                ]
            d_labels = [df_正.index.max(),df_负.index[j]]  # 一对冲正匹配记录的索引
            if len(df_正)>0:
                df_冲正 = df_冲正.append(df.loc[d_labels])
                df = df.drop(d_labels) # 记录后删除一对冲正匹配的无效流水行
                print(d_labels)
            elif len(df_正)==0:
                print(d_labels)
        print('剔除冲正后保留记录数:'+str(df.shape[0]))
        all_流水 = all_流水.append(df)
    elif file_list[i] in list(group_账户['Path']):
        if type1 in file_list[i]:
                df_account = pd.read_excel(file_list[i],encoding = "gbk",dtype = "object")
        elif type2 in file_list[i]:
                df_account = pd.read_csv(file_list[i], engine="python")
        df_account.columns = df_account.columns.str.strip()
        df_account['path'] = file_list[i]
        df_account = df_account.applymap(lambda x: re.sub('\t','',str(x)).strip())
        all_账户 = all_账户.append(df_account)
    elif file_list[i] in list(group_证件['Path']):
        if type1 in file_list[i]:
            df_identity = pd.read_excel(file_list[i],encoding = "gbk")
        elif type2 in file_list[i]:
            df_identity = pd.read_csv(file_list[i], engine="python")
        if df_identity.shape[1]==4 or df_identity.shape[1]==3:
            df_identity = df_identity.iloc[:,0:3]
            df_identity.columns = ['客户名称','证件类型','证件号码']
        df_identity.columns = df_identity.columns.str.strip()
        df_identity['path'] = file_list[i]
        df_identity = df_identity.applymap(lambda x: re.sub('\t','',str(x)).strip())
        all_证件 = all_证件.append(df_identity)
    elif file_list[i] in list(group_任务['Path']):
        if type1 in file_list[i]:
                df_task = pd.read_excel(file_list[i],encoding = "gbk",dtype = "object")
        elif type2 in file_list[i]:
                df_task = pd.read_csv(file_list[i], engine="python")
        df_task.columns = df_task.columns.str.strip()
        df_task['path'] = file_list[i]
        df_task = df_task.applymap(lambda x: re.sub('\t','',str(x)).strip())
        all_任务 = all_任务.append(df_task)
        
        
# all_流水.to_csv(output_path + "流水合并剔除冲正.csv",index = None)
print(all_流水.shape)
with pd.ExcelWriter(output_path + "剔除行.xlsx") as writer:
    df_冲正.to_excel(writer, sheet_name='冲正全',index = None)
    df_忽略行.to_excel(writer, sheet_name='忽略行',index = None)
print(df_冲正.shape,end = ' ')
print(df_忽略行.shape)

all_账户.to_excel(output_path + "账户合并.xlsx",index = None)
print(all_账户.shape)
all_证件.to_excel(output_path + "证件合并.xlsx",index = None)
print(all_证件.shape)

all_任务.to_excel(output_path + "任务合并.xlsx",index = None)

print('合并Done!')



# 卡号修正
# 汇总时已经填充了交易账号，如果交易卡号、交易账号是唯一对应关系，实际应该依赖交易账号填充对应的交易卡号
df = all_流水.sort_values(by = ['交易卡号','交易时间','交易金额'],ascending=[True,True,False]).reset_index().iloc[:,1:]  # 流水汇总的all_流水 更新索引

# 手工微调
df['交易卡号'] = df['交易卡号'].str.split('_', 1,expand = True).iloc[:,0] 
df['交易卡号'] = df['交易卡号'].str.split('-', 1,expand = True).iloc[:,0]

df = df[~((df['交易对手账卡号']=='6236681460012372435') & (df['对手户名']=='吴晶晶'))]
df = df[~((df['交易对手账卡号']=='6230780100017400236') & (df['对手户名']=='吴晶晶'))]
df = df[~((df['交易对手账卡号']=='6228411070343158413') & (df['对手户名']=='毛霓姣'))]
df = df[~((df['交易对手账卡号']=='6212261207008544728') & (df['对手户名']=='唐文强'))]
df = df[~((df['交易对手账卡号']=='6214830278991772') & (df['对手户名']=='傅洋'))]
df = df[~((df['交易对手账卡号']=='6228481259254490379') & (df['对手户名']=='孞凤芹'))]
df = df[~((df['交易对手账卡号']=='6214836124051679') & (df['对手户名']=='尹君东'))]
df = df[~((df['交易对手账卡号']=='6228480316044859064') & (df['对手户名']=='周志峰'))]
df = df[~((df['交易对手账卡号']=='6222031106003886704') & (df['对手户名']=='李宁江'))]
df = df[~((df['交易对手账卡号']=='6230366107007841602') & (df['对手户名']=='林克时'))]
df = df[~((df['交易对手账卡号']=='622908406965516613') & (df['对手户名']=='王琴'))]
df = df[~((df['交易对手账卡号']=='6226093711673251') & (df['对手户名']=='张彦青'))]
df = df[~((df['交易对手账卡号']=='6217003100015191884') & (df['对手户名']=='翁玉通'))]
df = df[~((df['交易对手账卡号']=='6222021208010967154') & (df['对手户名']=='曹文光'))]
df = df[~((df['交易对手账卡号']=='6217001440011169779') & (df['对手户名']=='吴刚'))]
df = df[~((df['交易对手账卡号']=='6212262201039744612') & (df['对手户名']=='许赵海'))]
df = df[~((df['交易对手账卡号']=='6228481829130673677') & (df['对手户名']=='梁增铜'))]
df = df[~((df['交易对手账卡号']=='6228480708847908272') & (df['对手户名']=='曾坜平'))]
df = df[~((df['交易对手账卡号']=='6230520040045389479') & (df['对手户名']=='徐志强农业'))]
df = df[~((df['交易对手账卡号']=='6230520040045389479') & (df['对手户名']=='志强徐'))]
df = df[~((df['交易对手账卡号']=='6230520040045389479') & (df['对手户名']=='强徐志'))]
df = df[~((df['交易对手账卡号']=='6228480028660500373') & (df['对手户名']=='蔡春生农业'))]
df = df[~((df['交易对手账卡号']=='6228480028660500373') & (df['对手户名']=='姜术梅'))]
df = df[~((df['交易对手账卡号']=='6228482188980863378') & (df['对手户名']=='贾立昌农业'))]
df = df[~((df['交易对手账卡号']=='6228482188980863378') & (df['对手户名']=='姜术梅'))]
df = df[~((df['交易对手账卡号']=='6228482188980863378') & (df['对手户名']=='徐志强'))]
df = df[~((df['交易对手账卡号']=='6228482188980863378') & (df['对手户名']=='李晓荣'))]
df = df[~((df['交易对手账卡号']=='6230522170015830174') & (df['对手户名']=='彭淼农业'))]
df = df[~((df['交易对手账卡号']=='6230522170015830174') & (df['对手户名']=='彭淼农'))]
df = df[~((df['交易对手账卡号']=='6228480188482304774') & (df['对手户名']=='40000'))]
df = df[~((df['交易对手账卡号']=='6228480188482304774') & (df['对手户名']=='术梅姜'))]
df = df[~((df['交易对手账卡号']=='6228480188482304774') & (df['对手户名']=='贾立昌'))]
df = df[~((df['交易对手账卡号']=='6230520040045672973') & (df['对手户名']=='贾立昌'))]
df = df[~((df['交易对手账卡号']=='6236680680002112456') & (df['对手户名']=='彭淼25'))]
df = df[~((df['交易对手账卡号']=='6236680680002112456') & (df['对手户名']=='彭淼建设'))]
df = df[~((df['交易对手账卡号']=='6236680680002112456') & (df['对手户名']=='彭淼建'))]
df = df[~((df['交易对手账卡号']=='6222032407001693723') & (df['对手户名']=='陈小平'))]
df = df[~((df['交易对手账卡号']=='6230580000291611205') & (df['对手户名']=='扬娇'))]
df = df[~((df['交易对手账卡号']=='6222031404000657552') & (df['对手户名']=='薛强'))]
df = df[~((df['交易对手账卡号']=='6236681840003035523') & (df['对手户名']=='程立中'))]
df = df[~((df['交易对手账卡号']=='6226621408346797') & (df['对手户名']=='10710'))]
df = df[~((df['交易对手账卡号']=='6228480616749798271') & (df['对手户名']=='黄健超'))]
df = df[~((df['交易对手账卡号']=='6214855912558265') & (df['对手户名']=='石双林6214855912558265'))]
df = df[~((df['交易对手账卡号']=='6217003490003134081') & (df['对手户名']=='陈业坤6217003490003134081'))]
df = df[~((df['交易对手账卡号']=='6217001930046828787') & (df['对手户名']=='张学凤'))]
df = df[~((df['交易对手账卡号']=='6215581511006905823') & (df['对手户名']=='黄新于'))]
df = df[~((df['交易对手账卡号']=='6217992320011203011') & (df['对手户名']=='彭淼邮政'))]
df = df[~((df['交易对手账卡号']=='6216692000001785939') & (df['对手户名']=='彭淼邮政'))]
df = df[~((df['交易对手账卡号']=='6222023100086849408') & (df['对手户名']=='1688'))]
df = df[~((df['交易对手账卡号']=='6216616304001985124') & (df['对手户名']=='程铁'))]
df = df[~((df['交易对手账卡号']=='6217231208008082228') & (df['对手户名']=='陈德云'))]
df = df[~((df['交易对手账卡号']=='6212253803003255649') & (df['对手户名']=='徐州'))]
df = df[~((df['交易对手账卡号']=='6222032115001835307') & (df['对手户名']=='范明洞'))]
df = df[~((df['交易对手账卡号']=='6228480382332171810') & (df['对手户名']=='吴晶晶'))]
df = df[~((df['交易对手账卡号']=='6228411070343158413') & (df['对手户名']=='毛雪英'))]
df = df[~((df['交易对手账卡号']=='6236681460020283863') & (df['对手户名']=='郑晶'))]
df = df[~((df['交易对手账卡号']=='6217683001665743') & (df['对手户名']=='林学军'))]
df = df[~((df['交易对手账卡号']=='6216696200014146741') & (df['对手户名']=='徐塞君'))]
df = df[~((df['交易对手账卡号']=='6228430389655538179') & (df['对手户名']=='朱宇峰'))]
df = df[~((df['交易对手账卡号']=='6222625180010584760') & (df['对手户名']=='吴智'))]
df = df[~((df['交易对手账卡号']=='6226225505214500') & (df['对手户名']=='粱梅枝'))]
df = df[~((df['交易对手账卡号']=='6222031510002379340') & (df['对手户名']=='曹租威'))]
df = df[~((df['交易对手账卡号']=='6222600760014199977') & (df['对手户名']=='彭邵忠'))]
df = df[~((df['交易对手账卡号']=='6222081208002341696') & (df['对手户名']=='朱宇峰'))]
df = df[~((df['交易对手账卡号']=='6222620760012812163') & (df['对手户名']=='刘逢仪'))]
df = df[~((df['交易对手账卡号']=='6230580000073960887') & (df['对手户名']=='吴晶晶'))]
df = df[~((df['交易对手账卡号']=='6217001860009855156') & (df['对手户名']=='荣庆'))]
df = df[~((df['交易对手账卡号']=='6217991340025370441') & (df['对手户名']=='庭维亮'))]
df = df[~((df['交易对手账卡号']=='6236680180004579885') & (df['对手户名']=='吴江岳'))]
df = df[~((df['交易对手账卡号']=='1809029001024407892') & (df['对手户名']=='陶速峰'))]

df['对手户名'] = df['对手户名'].replace('邓?婷','邓雲婷')

df['交易卡号'] = df['交易卡号'].replace('9080218010100100273702','6230361108126793851')
df['交易卡号'] = df['交易卡号'].replace('9070722010100100629197','6230361107126113383')
df['交易卡号'] = df['交易卡号'].replace('101004974664358','6230910199035009196')
df['交易卡号'] = df['交易卡号'].replace('891425925200012','6231310401002624344')
df['交易卡号'] = df['交易卡号'].replace('101014872560177','6230910399091964728')
df['交易卡号'] = df['交易卡号'].replace('1203285301114588770','6222031203012352131')
df['交易卡号'] = df['交易卡号'].replace('6228480136729683476','6228480136756146876')
df['交易卡号'] = df['交易卡号'].replace('6212261207015636822','6222033901009025812')
df['交易卡号'] = df['交易卡号'].replace('6217231208010997215','6222031202009735910')
df['交易卡号'] = df['交易卡号'].replace('6212261208012220446','6222032405004527799')
df['交易卡号'] = df['交易卡号'].replace('6222032407001751588','6222032405004527799')
df['交易卡号'] = df['交易卡号'].replace('6222031203009459055','6222031203012352131')
df['交易卡号'] = df['交易卡号'].replace('6212252003001207772','6222031203012352131')
df['交易卡号'] = df['交易卡号'].replace('6212261102044045659','6222031102003997369')
df['交易卡号'] = df['交易卡号'].replace('6212262012003068056','6222034000038894329')
df['交易卡号'] = df['交易卡号'].replace('6230523500010413171','6230523500010256174')
df['交易卡号']= df['交易卡号'].replace('6230520400151611175','6230520400230644379')
df['交易卡号']= df['交易卡号'].replace('6230520600111277575','6230520600115175270')
df['交易卡号']= df['交易卡号'].replace('6228412834555392179','6228412834555080279')
df['交易卡号']= df['交易卡号'].replace('6228481338992122179','6228481339108759177')
df['交易卡号']= df['交易卡号'].replace('6228480848817180179','6228480848874876370')
df['交易卡号']= df['交易卡号'].replace('6217566400013513025','6217566400014268967')
df['交易卡号']= df['交易卡号'].replace('6228480389555382479','6228480389558414071')
df['交易卡号']= df['交易卡号'].replace('6228412834553530978','6228412834553528873')
df['交易卡号']= df['交易卡号'].replace('6228482838833466775','6228482838835549172')
df['交易卡号']= df['交易卡号'].replace('6212261715009732589','6222031715003548195')
df['交易卡号']= df['交易卡号'].replace('6230522400019855970','6230522400022777278')
df['交易卡号']= df['交易卡号'].replace('6230520400215073172','6230520400228965976')
df['交易卡号']= df['交易卡号'].replace('6228482168970332279','6230522160020178776')
df['交易卡号']= df['交易卡号'].replace('6230520600113022375','6230520600121035179')
df['交易卡号']= df['交易卡号'].replace('101016427617930','6235040010000392941')
df['交易卡号']= df['交易卡号'].replace('6215581511004371473','6222031511000687882')
df['交易卡号']= df['交易卡号'].replace('6222032116002387470','6222032116003440724')
df['交易卡号']= df['交易卡号'].replace('6226097806695343','6214867811622233')
df['交易卡号']= df['交易卡号'].replace('6228481186717909978','6228481186719014272')
df['交易卡号']= df['交易卡号'].replace('6230521180000767670','6228481186719247674')
df['交易卡号']= df['交易卡号'].replace('6228410473068424765','6228410473082428263')
df['交易卡号']= df['交易卡号'].replace('6222032116003371184','6222032116003373776')
df['交易卡号']= df['交易卡号'].replace('6212260200195771890','6212260200126966692')
df['交易卡号']= df['交易卡号'].replace('6228480838701859277','6228480838944935371')
df['交易卡号']= df['交易卡号'].replace('6228480838922941474','6228480838944289472')
df['交易卡号']= df['交易卡号'].replace('6228480388090434779','6228480389559342370')
df['交易卡号']= df['交易卡号'].replace('6228460388011632679','6228460388011677872')
df['交易卡号']= df['交易卡号'].replace('6228480086981110371','6228480086984545375')
df['交易卡号']= df['交易卡号'].replace('6228480106038640970','6230520100032732071')
df['交易卡号']= df['交易卡号'].replace('6230522030022879277','6230522030022823671')
df['交易卡号']= df['交易卡号'].replace('6230560200029036998','6228450018014347379')
df['交易卡号']= df['交易卡号'].replace('6222031404000650912','6222031404000657404')
df['交易卡号']= df['交易卡号'].replace('6229920500122664639','6231330500573953887')
df['交易卡号']= df['交易卡号'].replace('6228481829123030372','6228481829134864074')
df['交易卡号']= df['交易卡号'].replace('6222081505000768444','6212261505002210299')
df['交易卡号']= df['交易卡号'].replace('6222081505000831580','6212261505002210299')
df['交易卡号']= df['交易卡号'].replace('6214835541788012','6214835543017519')
df['交易卡号']= df['交易卡号'].replace('6222031303001267652','6215581304003016327')
df['交易卡号']= df['交易卡号'].replace('6222081208009131769','6222081208009346276')
df['交易卡号']= df['交易卡号'].replace('6215583901002041186','6217233901005942562')
df['交易卡号']= df['交易卡号'].replace('6215581304002792654','6215581304002621408')
df['交易卡号']= df['交易卡号'].replace('6217231103001889176','6212251103000262439')
df['交易卡号']= df['交易卡号'].replace('6222081208003209231','6222081208009068573')
df['交易卡号']= df['交易卡号'].replace('6215581304002768415','6215581304002879279')
df['交易卡号']= df['交易卡号'].replace('6215591303000274705','6222031303001305932')
df['交易对手账卡号'] = df['交易对手账卡号'].replace('9080218010100100273702','6230361108126793851')
df['交易对手账卡号'] = df['交易对手账卡号'].replace('9070722010100100629197','6230361107126113383')
df['交易对手账卡号'] = df['交易对手账卡号'].replace('101004974664358','6230910199035009196')
df['交易对手账卡号'] = df['交易对手账卡号'].replace('891425925200012','6231310401002624344')
df['交易对手账卡号'] = df['交易对手账卡号'].replace('101014872560177','6230910399091964728')
df['交易对手账卡号'] = df['交易对手账卡号'].replace('1203285301114588770','6222031203012352131')
df['交易对手账卡号'] = df['交易对手账卡号'].replace('6228480136729683476','6228480136756146876')
df['交易对手账卡号'] = df['交易对手账卡号'].replace('6212261207015636822','6222033901009025812')
df['交易对手账卡号'] = df['交易对手账卡号'].replace('6217231208010997215','6222031202009735910')
df['交易对手账卡号'] = df['交易对手账卡号'].replace('6212261208012220446','6222032405004527799')
df['交易对手账卡号'] = df['交易对手账卡号'].replace('6222032407001751588','6222032405004527799')
df['交易对手账卡号'] = df['交易对手账卡号'].replace('6222031203009459055','6222031203012352131')
df['交易对手账卡号'] = df['交易对手账卡号'].replace('6212252003001207772','6222031203012352131')
df['交易对手账卡号'] = df['交易对手账卡号'].replace('6212261102044045659','6222031102003997369')
df['交易对手账卡号'] = df['交易对手账卡号'].replace('6212262012003068056','6222034000038894329')
df['交易对手账卡号'] = df['交易对手账卡号'].replace('6230523500010413171','6230523500010256174')
df['交易对手账卡号']= df['交易对手账卡号'].replace('6230520400151611175','6230520400230644379')
df['交易对手账卡号']= df['交易对手账卡号'].replace('6230520600111277575','6230520600115175270')
df['交易对手账卡号']= df['交易对手账卡号'].replace('6228412834555392179','6228412834555080279')
df['交易对手账卡号']= df['交易对手账卡号'].replace('6228481338992122179','6228481339108759177')
df['交易对手账卡号']= df['交易对手账卡号'].replace('6228480848817180179','6228480848874876370')
df['交易对手账卡号']= df['交易对手账卡号'].replace('6217566400013513025','6217566400014268967')
df['交易对手账卡号']= df['交易对手账卡号'].replace('6228480389555382479','6228480389558414071')
df['交易对手账卡号']= df['交易对手账卡号'].replace('6228412834553530978','6228412834553528873')
df['交易对手账卡号']= df['交易对手账卡号'].replace('6228482838833466775','6228482838835549172')
df['交易对手账卡号']= df['交易对手账卡号'].replace('6212261715009732589','6222031715003548195')
df['交易对手账卡号']= df['交易对手账卡号'].replace('6230522400019855970','6230522400022777278')
df['交易对手账卡号']= df['交易对手账卡号'].replace('6230520400215073172','6230520400228965976')
df['交易对手账卡号']= df['交易对手账卡号'].replace('6228482168970332279','6230522160020178776')
df['交易对手账卡号']= df['交易对手账卡号'].replace('6230520600113022375','6230520600121035179')
df['交易对手账卡号']= df['交易对手账卡号'].replace('101016427617930','6235040010000392941')
df['交易对手账卡号']= df['交易对手账卡号'].replace('6215581511004371473','6222031511000687882')
df['交易对手账卡号']= df['交易对手账卡号'].replace('6222032116002387470','6222032116003440724')
df['交易对手账卡号']= df['交易对手账卡号'].replace('6226097806695343','6214867811622233')
df['交易对手账卡号']= df['交易对手账卡号'].replace('6228481186717909978','6228481186719014272')
df['交易对手账卡号']= df['交易对手账卡号'].replace('6230521180000767670','6228481186719247674')
df['交易对手账卡号']= df['交易对手账卡号'].replace('6228410473068424765','6228410473082428263')
df['交易对手账卡号']= df['交易对手账卡号'].replace('6222032116003371184','6222032116003373776')
df['交易对手账卡号']= df['交易对手账卡号'].replace('6212260200195771890','6212260200126966692')
df['交易对手账卡号']= df['交易对手账卡号'].replace('6228480838701859277','6228480838944935371')
df['交易对手账卡号']= df['交易对手账卡号'].replace('6228480838922941474','6228480838944289472')
df['交易对手账卡号']= df['交易对手账卡号'].replace('6228480388090434779','6228480389559342370')
df['交易对手账卡号']= df['交易对手账卡号'].replace('6228460388011632679','6228460388011677872')
df['交易对手账卡号']= df['交易对手账卡号'].replace('6228480086981110371','6228480086984545375')
df['交易对手账卡号']= df['交易对手账卡号'].replace('6228480106038640970','6230520100032732071')
df['交易对手账卡号']= df['交易对手账卡号'].replace('6230522030022879277','6230522030022823671')
df['交易对手账卡号']= df['交易对手账卡号'].replace('6230560200029036998','6228450018014347379')
df['交易对手账卡号']= df['交易对手账卡号'].replace('6222031404000650912','6222031404000657404')
df['交易对手账卡号']= df['交易对手账卡号'].replace('6229920500122664639','6231330500573953887')
df['交易对手账卡号']= df['交易对手账卡号'].replace('6228481829123030372','6228481829134864074')
df['交易对手账卡号']= df['交易对手账卡号'].replace('6222081505000768444','6212261505002210299')
df['交易对手账卡号']= df['交易对手账卡号'].replace('6222081505000831580','6212261505002210299')
df['交易对手账卡号']= df['交易对手账卡号'].replace('6214835541788012','6214835543017519')
df['交易对手账卡号']= df['交易对手账卡号'].replace('6222031303001267652','6215581304003016327')
df['交易对手账卡号']= df['交易对手账卡号'].replace('6222081208009131769','6222081208009346276')
df['交易对手账卡号']= df['交易对手账卡号'].replace('6215583901002041186','6217233901005942562')
df['交易对手账卡号']= df['交易对手账卡号'].replace('6215581304002792654','6215581304002621408')
df['交易对手账卡号']= df['交易对手账卡号'].replace('6217231103001889176','6212251103000262439')
df['交易对手账卡号']= df['交易对手账卡号'].replace('6222081208003209231','6222081208009068573')
df['交易对手账卡号']= df['交易对手账卡号'].replace('6215581304002768415','6215581304002879279')
df['交易对手账卡号']= df['交易对手账卡号'].replace('6215591303000274705','6222031303001305932')

df['交易卡号']= df['交易卡号'].replace('103947006800018','6223687310841210434')
df['交易卡号']= df['交易卡号'].replace('11000504482600000001','6214467873197188474')
df['交易卡号']= df['交易卡号'].replace('121766634600010','6214467873131304146')
df['交易卡号']= df['交易卡号'].replace('4682035910003238','6214855912558265')
df['交易卡号']= df['交易卡号'].replace('6214830273485036','6214857216248072')
df['交易卡号']= df['交易卡号'].replace('5240115712686869','6214866124686869')
df['交易卡号']= df['交易卡号'].replace('160000000894384','6214391880035962193')
df['交易卡号']= df['交易卡号'].replace('101007005015630','6228580199029260279')
df['交易卡号']= df['交易卡号'].replace('220360191010363942','6230388800024258941')
df['交易卡号']= df['交易卡号'].replace('101012300502118','6230910199032981512')
df['交易卡号']= df['交易卡号'].replace('101014881848187','6230910799055059458')
df['交易卡号']= df['交易卡号'].replace('101013050986481','6230910799062780419')
df['交易对手账卡号']= df['交易对手账卡号'].replace('21*****90','215500690')
df['交易对手账卡号']= df['交易对手账卡号'].replace('24*****33','243300133')
df['交易对手账卡号']= df['交易对手账卡号'].replace('21*****93','215500693')
df['交易对手账卡号']= df['交易对手账卡号'].replace('22*****12','220800312')
df['交易对手账卡号']= df['交易对手账卡号'].replace('1107095201003626697','6215581107004822894')
df['交易对手账卡号']= df['交易对手账卡号'].replace('6228481319044585779','6228481319048180072')
df['交易对手账卡号']= df['交易对手账卡号'].replace('6228481829123039175','6228481829131454879')
df['交易对手账卡号']= df['交易对手账卡号'].replace('6228480848872393972','6228480848872581675')
df['交易对手账卡号']= df['交易对手账卡号'].replace('6228481198931565072','6228481199093759172')
df['交易对手账卡号']= df['交易对手账卡号'].replace('6228483618634108472','6228483618634624478')
df['交易对手账卡号']= df['交易对手账卡号'].replace('6230520450115585577','6230520450115936473')
df['交易对手账卡号']= df['交易对手账卡号'].replace('6230522320033203579','6230522320030003675')

  

df['交易户名'][df['交易卡号']=='6230361104131659666'] = df['交易户名'][df['交易卡号']=='6230361104131659666'].replace('','范金锋')
# df['交易户名'][df['交易卡号']=='6212261202035906396'] = df['交易户名'][df['交易卡号']=='6212261202035906396'].replace('','张一帆')
# df['交易户名'][df['交易卡号']=='6221804330000582705'] = df['交易户名'][df['交易卡号']=='6221804330000582705'].replace('','曾文强')
df = df[df['交易对手账卡号']!='62220340000024751625']
# list1 = ['6228480848874876370','6228480078248493377'] 
# df = df[~df['交易卡号'].isin(list1)]
df = df[df['交易卡号']!='160000010525495']
df = df[df['交易卡号']!='6228480708833488578']
print('手工微调后',end = '')
print(df.shape)




df_账号 = df[(df['交易卡号']!=df['交易账号']) & (df['交易账号']!='')] #  提取交易卡号、交易账号的对应关系表
df_a1 = df_账号.groupby(['交易账号','交易卡号']).size().reset_index()
df_a2 = df_a1.groupby(['交易账号'],as_index = False).size()
card = df_a1[df_a1['交易账号'].isin(df_a2[df_a2==1].index)]
card.to_excel(output_path + "卡号账号唯一对应.xlsx",index = None)
print('账号卡号对应Done!')
dfs0 = pd.merge(df,card,how = 'left',on='交易账号')
print(dfs0.columns)
dfs0['交易卡号_x'][(dfs0['交易卡号_x']!=dfs0['交易卡号_y']) & (dfs0['交易卡号_y'].notnull())]=dfs0['交易卡号_y'][(dfs0['交易卡号_x']!=dfs0['交易卡号_y']) & (dfs0['交易卡号_y'].notnull())]
all_流水卡修正 = dfs0.iloc[:,:34]
all_流水卡修正.columns = all_流水.columns
# all_流水卡修正.to_csv(output_path + "流水卡号.csv",index = None)
print(all_流水卡修正.shape)



# 户名填充
# 方法一：依赖账户开户信息表汇总的all_账户
df_账户 = all_账户[all_账户['账户开户名称'] != '']
df_账户['交易卡号'] = df_账户['交易卡号'].str.split('_', 1,expand = True).iloc[:,0]   # 账户开户信息中卡号去掉_后缀
df_账户['交易卡号'] = df_账户['交易卡号'].str.split('-', 1,expand = True).iloc[:,0]  # 账户开户信息中卡号去掉-后缀
#  执行了手动修正卡号则屏蔽
df_流水 = all_流水卡修正  #  df_流水 是手动修正卡号后的流水汇总 ，不需要手动就用账号卡号唯一关系修正的 all_流水卡修正 

dfs = pd.merge(df_流水,df_账户.drop_duplicates(subset='交易卡号'),how = 'left',on = '交易卡号').iloc[:,:(df_流水.shape[1]+2)]
dfs['交易户名'][dfs['账户开户名称'].notnull()] = dfs['账户开户名称'][dfs['账户开户名称'].notnull()]
dfs['交易证件号码'][dfs['开户人证件号码'].notnull()] = dfs['开户人证件号码'][dfs['开户人证件号码'].notnull()]
df = dfs.iloc[:,:34]
df.columns = df_流水.columns
print(df.shape)
# 方法二：依赖交易对手卡号、对手户名的唯一对应关系
df_户名 = df[(df['交易对手账卡号'].isin(df['交易卡号'][df['交易户名']==''])) & (df['对手户名']!='')]  # 卡号为主账号且作为对手卡号出现时对手户名非空，作为本方卡号出现时户名缺失
df_a1 = df_户名.groupby(['交易对手账卡号','对手户名']).size().reset_index()
df_a2 = df_a1.groupby(['交易对手账卡号'],as_index=False).size()
account2 = df_a1[df_a1['交易对手账卡号'].isin(df_a2[df_a2==1].index)]
account2.to_excel(output_path + "卡号户名唯一对应.xlsx",index = None)
print('卡号户名对应Done!')
dfs2 = pd.merge(df,account2,how = 'left',left_on='交易卡号',right_on='交易对手账卡号')
print(dfs2.columns)
dfs2['交易户名'][dfs2['对手户名_y'].notnull()] = dfs2['对手户名_y'][dfs2['对手户名_y'].notnull()]
df = dfs2.iloc[:,:34]
df.columns = df_流水.columns
print(df.shape)
# # 方法三：依赖手工整理的银行卡号、姓名唯一对应表格
# account3 = pd.read_excel('E:\\7.3徐州网赌\\银行卡人员信息整理(提供给公司).xls',sheet_name = '去重',encoding = 'gbk')
# dfs3 = pd.merge(df,account3.drop_duplicates(subset='银行卡号'),how = 'left',left_on='交易卡号',right_on='银行卡号')
# dfs3['交易户名'][dfs3['姓名'].notnull()] = dfs3['姓名'][dfs3['姓名'].notnull()]
# df = dfs3.iloc[:,:34]
# df.columns = df_流水.columns
# print(df.shape)
df_流水户名 = df
df_流水户名.to_csv(output_path +"流水户名.csv",index = None)
print('流水户名Done!')

# #--------------------------------------------------保留7月以后---------------------------------------------------------#
# df = df[(df['交易时间'].str[0:4].isin(['2020']) ) & (df['交易时间'].str[5:7].isin(['07','08','09','10','11']) )]
# #--------------------------------------------------保留7月以后---------------------------------------------------------#

df_S1 = df.groupby('交易卡号').size().reset_index()
df_S2 = pd.DataFrame(columns = ['交易卡号','Size','False','True'])
df_S3 = pd.DataFrame(columns = ['交易卡号','交易户名','证件号码','夜间交易次数','最早时间','最晚时间','余额','总和','净和','入向','出向','入向对手数','出向对手数','收付检验起始值','Path'])
for i in df_S1.index:
    df_every = df.groupby('交易卡号').get_group(df_S1['交易卡号'][i])
    bool_S = df_every.duplicated(subset = ['交易卡号','交易时间','交易金额','交易余额','收付标志','交易对手账卡号','交易币种'],keep = 'last').value_counts()
    df_S2.loc[i] = {'交易卡号':df_every['交易卡号'].iloc[0],
                        'Size':df_every.shape[0],
                        'False':bool_S[0],
                        'True':bool_S.map(lambda x:0 if len(bool_S)==1 else bool_S[1]).values[0]
                    }
    df_every = df_every.drop_duplicates(subset = ['交易卡号','交易时间','交易金额','交易余额','收付标志','交易对手账卡号','交易币种'],keep = 'last')
    df_every.to_excel(output_path+"资金流水\\" + str(df_every['交易户名'].iloc[0])+str(df_every['交易卡号'].iloc[0])+".xlsx",index = None)  # 拆表
    df_night = df_every[df_every['交易时间'].str[11:13].isin(['00','01','02','03','04','05','06'])]
    df_S3.loc[i] = {'交易卡号':df_every['交易卡号'].iloc[0],
                    '交易户名':df_every['交易户名'].iloc[0],
                    '证件号码':df_every['交易证件号码'].iloc[0],
                    '夜间交易次数':len(df_night),
                    '最早时间':df_every['交易时间'].min(),
                    '最晚时间':df_every['交易时间'].max(),
                    '余额':df_every['交易余额'].loc[df_every.index.max()],
                    '总和':df_every['交易金额'].sum(),
                    '净和':df_every['交易金额'][df_every['收付标志']=='进'].sum()-df_every['交易金额'][df_every['收付标志']=='出'].sum(),
                    '入向':df_every['交易金额'][df_every['收付标志']=='进'].sum(),
                    '出向':df_every['交易金额'][df_every['收付标志']=='出'].sum(),
                    '入向对手数':len(df_every['交易对手账卡号'][df_every['收付标志']=='进'].drop_duplicates()),
                    '出向对手数':len(df_every['交易对手账卡号'][df_every['收付标志']=='出'].drop_duplicates()),
                    '收付检验起始值':df_every['交易余额'].iloc[0]+df_every.apply(lambda x: x['交易金额'] if x['收付标志']=='出' else -x['交易金额'],axis=1).iloc[0],
                    'Path':'\n'.join(df_every['path'].drop_duplicates().tolist())
                   }
    print(i,end = ' ')
df_S = pd.merge(df_S3,df_S2,how = 'left',on = '交易卡号')
df_账户['开户网点'][df_账户['开户网点']==''] = df_账户['账号开户银行'][df_账户['开户网点']=='']  # 网点为空用账户开户银行填充
dfs_账户银行 = pd.merge(df_S,df_账户.drop_duplicates(subset='交易卡号'),how = 'left',on = '交易卡号')
df_S['开户网点'] = dfs_账户银行['开户网点']
df_S['账户状态'] = dfs_账户银行['账户状态']
df_S.to_excel(output_path + "交易统计.xlsx",index = None)
# 统计完了，按照六大要素+币种 删除重复项（因为手动改了-101与-102的双向交易的卡号后缀，-101出-102进这类记录会造成自实体链接，不影响）
all_流水完整 = df.drop_duplicates(subset = ['交易卡号','交易时间','交易金额','交易余额','收付标志','交易对手账卡号','交易币种'],keep = 'last')
all_流水完整.to_csv(output_path + "流水完整.csv",index = None)
print(all_流水完整.shape)